In [8]:
!pip install -q pandas

ERROR: snowflake-ml-python 1.0.10 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.2.1 which is incompatible.
ERROR: snowflake-ml-python 1.0.10 has requirement snowflake-snowpark-python<2,>=1.5.1, but you'll have snowflake-snowpark-python 1.5.0 which is incompatible.
ERROR: snowflake-connector-python 3.7.1 has requirement urllib3<2.0.0,>=1.21.1; python_version < "3.10", but you'll have urllib3 2.1.0 which is incompatible.
ERROR: botocore 1.34.51 has requirement urllib3<1.27,>=1.25.4; python_version < "3.10", but you'll have urllib3 2.1.0 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from snowflake.snowpark.session import Session
import pandas as pd
import subprocess

In [2]:
from snowflake.snowpark.session import Session

connection_params = {
    'user': 'MANISH',
    'password': 'PASSWORD@2023',
    'account': 'ug94937.us-east4',
    'warehouse': 'FOSFOR_FDC',
    'database': 'FDC_DATA_MANISH',
    'schema': 'PUBLIC',
    'role': 'MANISH'
}
session = Session.builder.configs(connection_params).create()
session.sql('use warehouse FOSFOR_FDC;').collect()
session.sql('use database FDC_DATA_MANISH;').collect()
session.sql('use schema FDC_DATA_MANISH.PUBLIC;').collect()

df =  pd.read_csv("/notebooks/notebooks/customer_data.csv")
df_model=session.createDataFrame(
        df.values.tolist(),
        schema=df.columns.tolist())
df_model.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.TABLE_NAME")

FileNotFoundError: [Errno 2] No such file or directory: 'snowsql'